In [9]:
import pandas as pd
import numpy as np

In [7]:
survey_raw = pd.read_excel("../data/raw/SurveyReport-7464592-02-21-2022-T150410.923.xlsx",
    sheet_name="Dados não tratados")

c:\Users\Daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
#remover concentimento e "Variável Personalizada"
survey_f00 = survey_raw.drop(survey_raw.columns[
    np.r_[6:14, #Referência externa, Variável Personalizada(1 a 5), email, lista
    25:29], # termos de concentimento
], 
    axis=1)

In [12]:
def first_valid(x):
#https://www.semicolonworld.com/question/58645/first-non-null-value-per-row-from-a-list-of-pandas-columns
    if x.first_valid_index() is None:
        return None
    else:
        return x[x.first_valid_index()]

locais_resid = survey_f00.loc[:, survey_f00.columns.str.startswith("Qual o seu programa de residência?")]
#np.where(~np.isnan(locais_resid.values))

    
local_residencia = locais_resid.apply(first_valid, axis=1)

survey_f01 = survey_f00.drop(survey_f00.columns
                             [survey_f00.columns.str.startswith("Qual o seu programa de residência?")],
                             axis=1)
survey_f01["Qual o seu programa de residência?"] = local_residencia

In [13]:
#filtra somente para residentes que completaram, retirando colunas totalmente vazias
residentes_completed = survey_f01.loc[(survey_f01["Status de resposta"]=="Completed") &
               (survey_f01["Qual a sua posição na residência?"] !=
                "Coordenador da Residência")]

residentes_completed = residentes_completed.dropna(axis=1, how="all")
residentes_completed = residentes_completed.reset_index(drop=True)

In [14]:
#Remover questoes que residentes nao deveriam responder
residentes_filter = residentes_completed.drop(residentes_completed.columns[
    residentes_completed.columns.str.startswith("Seu programa de residência tem estágio curricular")],axis=1)

residentes_filter = residentes_filter.drop(residentes_filter.columns[
    residentes_filter.columns.str.startswith("Seu programa de residência oferece alguma das capacitações abaixo relacionado")],axis=1)

#retira questoes sobre retorno da participacao
residentes_filter = residentes_filter.iloc[:, np.r_[0:62, 64:65]]

In [15]:
columns_id = [
    0,  #'ID de resposta',
    1,  #'Status de resposta',
    3,  #'Timestamp (mm/dd/yyyy)',
    62, #'Qual o seu programa de residência?',
    7,  #'Região', #Qual o estado de seu Programa de Residência Médica?
    15, #'Qual o seu gênero?',
    16, #'Qual é a sua data de nascimento?',
    17  #'Considerando as opções abaixo, como você classificaria sua cor ou raça?'
]


questoes_seguranca = [ #list(range(43, 62)
    'Qual o seu nível de segurança em relação a INTUBAÇÃO OROTRAQUEAL? ',
    'Qual o seu nível de segurança em relação ao ATENDIMENTO DE PCR?',
    'Qual o seu nível de segurança em relação ao ATENDIMENTO DE PACIENTE POLITRAUMATIZADO?',
    'Qual o seu nível de segurança em relação à passagem de ACESSO VENOSO CENTRAL?',
    'Qual o seu nível de segurança em relação à passagem de CATETER DE SHILLEY?',
    'Qual o seu nível de segurança em relação à passagem de ACESSO VENOSO PERIFÉRICO?',
    'Qual o seu nível de segurança em relação à realização de TORACOCENTESE?',
    'Qual o seu nível de segurança em relação à realização de PARACENTESE?',
    'Qual o seu nível de segurança em relação à realização de PUNÇÃO LOMBAR?',
    'Qual o seu nível de segurança em relação à colocação de DRENO DE TÓRAX?',
    'Qual o seu nível de segurança em relação à realização de BLOQUEIO NERVOSO PERIFÉRICO?',
    'Qual o seu nível de segurança em relação à realização de CARDIOVERSÃO?',
    'Qual o seu nível de segurança em relação à realização de PERICARDIOCENTESE?',
    'Qual o seu nível de segurança em relação à realização de PUNÇÃO SUPRAPÚBICA?',
    'Qual o seu nível de segurança em relação à instalação de ACESSO INTRAÓSSEO?',
    'Qual o seu nível de segurança em relação à realização de DRENAGEM DE ABSCESSO?',
    'Qual o seu nível de segurança em relação à realização de CRICOTIREOIDOSTOMIA?',
    'Qual o seu nível de segurança em relação à passagem de LINHA ARTERIAL/PA INVASIVA?',
    'Qual o seu nível de segurança em relação à coleta de GASOMETRIA ARTERIAL?'
]

questoes_escolha = [ #list(range(20, 26))
    'Quanto cada uma das variáveis abaixo influenciou em sua escolha para o seu atual Programa de Residência de Medicina de Emergência? - Cidade do Programa',
    'Quanto cada uma das variáveis abaixo influenciou em sua escolha para o seu atual Programa de Residência de Medicina de Emergência? - Qualidade do Programa',
    'Quanto cada uma das variáveis abaixo influenciou em sua escolha para o seu atual Programa de Residência de Medicina de Emergência? - Presença de preceptores emergencistas',
    'Quanto cada uma das variáveis abaixo influenciou em sua escolha para o seu atual Programa de Residência de Medicina de Emergência? - Facilidade de acesso',
    'Quanto cada uma das variáveis abaixo influenciou em sua escolha para o seu atual Programa de Residência de Medicina de Emergência? - Hospitais conveniados ',
    'Quanto cada uma das variáveis abaixo influenciou em sua escolha para o seu atual Programa de Residência de Medicina de Emergência? - Fluxo de pacientes'
]

questoes_satisfacao = [ #list(range(35, 43))
    'Em relação ao seu programa de residência, como você avalia: - Sua satisfação',
    'Em relação ao seu programa de residência, como você avalia: - Disponibilidade dos preceptores',
    'Em relação ao seu programa de residência, como você avalia: - Locais de Prática',
    'Em relação ao seu programa de residência, como você avalia: - Atividades Teóricas',
    'Em relação ao seu programa de residência, como você avalia: - Incentivo a pesquisa',
    'Em relação ao seu programa de residência, como você avalia: - Nº de Atendimentos',
    'Em relação ao seu programa de residência, como você avalia: - Disponibilidade de material',
    'Em relação ao seu programa de residência, como você avalia: - Diversidade de doenças',
]

questoes_morte_encefalica = [ #list(range(29, 35))
    'Seu programa de residência oferece alguma das capacitações abaixo para DETERMINAÇÃO DE MORTE ENCEFÁLICA? (marque todos que são oferecidos) - Aula teórica específica sobre o tema',
    'Seu programa de residência oferece alguma das capacitações abaixo para DETERMINAÇÃO DE MORTE ENCEFÁLICA? (marque todos que são oferecidos) - Estágio curricular específico sobre o tema',
    'Seu programa de residência oferece alguma das capacitações abaixo para DETERMINAÇÃO DE MORTE ENCEFÁLICA? (marque todos que são oferecidos) - Curso específico sobre o tema',
    'Seu programa de residência oferece alguma das capacitações abaixo para DETERMINAÇÃO DE MORTE ENCEFÁLICA? (marque todos que são oferecidos) - Simulação específica sobre o tema ',
    'Seu programa de residência oferece alguma das capacitações abaixo para DETERMINAÇÃO DE MORTE ENCEFÁLICA? (marque todos que são oferecidos) - Treinamento beira-leito sobre o tema',
    'Seu programa de residência oferece alguma das capacitações abaixo para DETERMINAÇÃO DE MORTE ENCEFÁLICA? (marque todos que são oferecidos) - Não oferece nenhum treinamento específico '
]


In [16]:
seguranca_residentes = residentes_filter.iloc[:, [0, 1, 3, 62, 7, 15, 16, 17, 19] + list(range(43, 62))]

escolha_residentes = residentes_filter.iloc[:, [0, 1, 3, 62, 7, 15, 16, 17, 19] + list(range(20, 26))]

satisfacao_residentes = residentes_filter.iloc[:, [0, 1, 3, 62, 7, 15, 16, 17, 19] + list(range(35, 43))]

me_residentes = residentes_filter.iloc[:, [0, 1, 3, 62, 7, 15, 16, 17, 19] + list(range(29, 35))]